Necessary functions to check the sign condition, compute the Gram matrix, unimodular matrix, and $\alpha^2$.

In [21]:
def sign_cond(alpha_0,beta_0,gamma_0):
    s1 = alpha_0 + beta_0 + gamma_0
    s2 = alpha_0*beta_0 + beta_0*gamma_0 + gamma_0*alpha_0
    s3 = alpha_0*beta_0*gamma_0
    return s1, s2, s3


In [22]:
def factor_coeficient_of_alpha2(alpha_0):
    alpha_poly = alpha_0.lift()
    R = alpha_poly.parent()
    rho = R.gen()

    #  m, n, p from m*rho^2 + n*rho + p
    m = alpha_poly.coefficient(2)
    n = alpha_poly.coefficient(1)
    p = alpha_poly.coefficient(0)

    # Factor coefficients
    m_factored = m.factor()
    n_factored = n.factor()
    p_factored = p.factor()

    print("m = ",m_factored)
    print("n = ",n_factored)
    print("p = ",p_factored)


In [23]:
def to_transition_matrix(base_old, base_new, rho):
    #  [1, rho, rho^2]
    def to_vec(expr):
        expr = SR(expr).expand()  
        return vector(SR, [
            expr.subs(rho=0),
            expr.coefficient(rho),
            expr.coefficient(rho^2)
        ])
    
    B = matrix(SR, [to_vec(b) for b in base_old]).transpose()
    Bp = matrix(SR, [to_vec(bn) for bn in base_new]).transpose()

    U_cols = [B.solve_right(col) for col in Bp.columns()]
    U = matrix(SR, U_cols).transpose()
    return U.apply_map(lambda x: x.simplify_rational())

In [24]:
def rewrite_alpha0_and_compute_norm_psi(alpha_0,get_rho_permutations):
    alpha_poly = alpha_0.lift()
    R = alpha_poly.parent()
    rrho = R.gen()

    coeffs = [alpha_poly.coefficient(i) for i in range(alpha_poly.degree() + 1)]

    common_factor = coeffs[0]
    for c in coeffs[1:]:
        common_factor = gcd(common_factor, c)

    print("k = ",common_factor.factor())

    simplified_coeffs = [c // common_factor for c in coeffs]
    psi = sum(simplified_coeffs[i] * rrho^i for i in range(len(simplified_coeffs)))

    print("psi = ",psi)
    rho1, rho2, rho3 = get_rho_permutations(rho)
    psi1 = psi.subs(r=rho1)
    psi2 = psi.subs(r=rho2)
    psi3 = psi.subs(r=rho3)
    norm = psi1 * psi2 * psi3
    return norm

In [25]:
def compute_gram_matrix(df, rho, basis, get_rho_permutations):
    rho1, rho2, rho3 = get_rho_permutations(rho)
    x1, y1, z1 = basis(rho1)
    x2, y2, z2 = basis(rho2)
    x3, y3, z3 = basis(rho3)

    M_alpha = Matrix([[x2^2 - y2^2, x3^2 - y3^2],
                      [y2^2 - z2^2, y3^2 - z3^2]])
    alpha_0 = M_alpha.det()

    M_beta = Matrix([[x3^2 - y3^2, x1^2 - y1^2],
                     [y3^2 - z3^2, y1^2 - z1^2]])
    beta_0 = M_beta.det()

    M_gamma = Matrix([[x1^2 - y1^2, x2^2 - y2^2],
                      [y1^2 - z1^2, y2^2 - z2^2]])
    gamma_0 = M_gamma.det()
    a = alpha_0*x1^2 + beta_0*x2^2 + gamma_0*x3^2       #||Tx||^2 = ||Ty||^2 = ||Tz||^2
    b = alpha_0*x1*y1 + beta_0*x2*y2 + gamma_0*x3*y3    #<Tx,Ty>
    c = alpha_0*z1*x1 + beta_0*z2*x2 + gamma_0*z3*x3    #<Tz,Tx>
    d = alpha_0*y1*z1 + beta_0*y2*z2 + gamma_0*y3*z3    #<Ty,Tz>
    s1 = sign_cond(alpha_0, beta_0, gamma_0)[0]
    s2 = sign_cond(alpha_0, beta_0, gamma_0)[1]
    s3 = sign_cond(alpha_0, beta_0, gamma_0)[2]
    return s1,s2,s3,a,b,c,d,alpha_0



Shank's  simplest cubic fields, good basis   $\left\{\dfrac{1 + \rho + \rho^2}{3},\rho,\rho+\rho^2\right\}$

In [26]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    1,
    rho,
    (1+rho+rho^2)/3
] # we know that this is an integral basis.
good_basis = [
    (1+rho+rho^2)/3,
    rho,
    rho+rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df = r^3 - n*r^2 - (n + 3)*r - 1
L.<rho> = K.quotient(df)

def shank_basis(r):
    x = (1 + r + r^2)/3
    y = r
    z = r + r^2
    return x, y, z
def shank_permutation(rho):
    rho1 = rho
    rho2 = -1 / (1 + rho1)
    rho3 = -1/(1+ rho2)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=shank_basis, get_rho_permutations=shank_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=shank_permutation).lift()).factor())    


transition matrix = 
[ 0  0 -1]
[ 0  1  0]
[ 1  0  3]
determinant =  1

s1 = (8/9) * (n^2 + 3*n + 9) * (n^2 + 9/2*n + 6)
s2 = (7/81) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 236/7*n^4 + 429/7*n^3 + 234/7*n^2 - 459/7*n - 81)
s3 = (7/729) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 180/7*n^4 + 135/7*n^3 - 108/7*n^2 - 81/7*n - 243/7)

a = (1/9) * (n^2 + 3*n + 3) * (n^2 + 3*n + 9)
b = (-1/27) * (n^2 + 3*n + 9) * (n^2 + 9*n + 9)
c = (1/27) * (n^2 - 3*n - 9) * (n^2 + 3*n + 9)
d = (2/3) * (n^2 + 3*n + 9)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-2/3) * (n^3 + 47/6*n^2 + 49/2*n + 57/2)
n =  (2/3) * (n^4 + 23/3*n^3 + 151/6*n^2 + 69/2*n + 21/2)
p =  (7/9) * (n^4 + 62/7*n^3 + 256/7*n^2 + 543/7*n + 486/7)

k =  1
psi =  (-2/3*n^3 - 47/9*n^2 - 49/3*n - 19)*r^2 + (2/3*n^4 + 46/9*n^3 + 151/9*n^2 + 23*n + 7)*r + 7/9*n^4 + 62/9*n^3 + 256/9*n^2 + 181/3*n + 54
norm psi =  (7/729) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 180/7*n^4 + 135/7*n^3 - 108/7*n^2 - 81/7*n - 243/7)


Washington’s cyclic cubic fields, $n$ even, good basis $\left\{\rho,\dfrac{\rho^2-1}{n-1}-\rho,\rho^2\right\}$

In [27]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    1,
    rho,
    (rho^2 - 1)/(n - 1)
] # we know that this is an integral basis.
good_basis = [
    rho,
    (rho^2 - 1)/(n - 1) - rho,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis(r):
    x = r
    y = (r^2-1)/(n-1)-r 
    z = r^2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=washington_basis, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=washington_permutation).lift()).factor()) 


transition matrix = 
[    0     0     1]
[    1    -1     0]
[    0     1 n - 1]
determinant =  1

s1 = (-2) * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 2*n^3 + 9/2*n^2 - 13/2*n + 9/2)
s2 = (n - 1)^4 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^2 * (n^8 - 4*n^7 + 12*n^6 - 26*n^5 + 42*n^4 - 52*n^3 + 49*n^2 - 30*n + 11)
s3 = (-1) * (n - 1)^6 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^3

a = (-1) * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 - n + 3) * (n^2 + 3)
b = (-1) * n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)
c = n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)
d = (-1) * n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)

alpha2 = m*rho^2 + n*rho + p where 
m =  (2) * (n - 1)^2 * (n^2 + 3) * (n^3 - 7/2*n^2 + 5*n - 2)
n =  (-2) * (n - 1)^3 * (n^2 + 3) * (n^5 - 9/2*n^4 + 21/2*n^3 - 31/2*n^2 + 16*n - 8)
p =  (-1) * (n - 1)^2 * (n^2 + 3) * (n^6 - 4*n^5 + 10*n^4 - 21*n^3 + 35*n^2 - 32*n + 13)

k =  (n - 1)^2 * (n^2 + 3)
psi =  (2*n^3 - 7*n^2 + 10*n - 4)*r^2 + (-2*n^6 + 11*n^5 - 30*n^4 + 52*n^3 - 63*n^2 + 48*n - 16)*r - n^6

Washington’s cyclic cubic fields, $n$ odd, good basis $\left\{\rho^2,\dfrac{n-2}{2n-2}\rho^2+\dfrac{\rho}{2}+\dfrac{1}{2n-2},\dfrac{\rho^2+\rho}{2}\right\}$

In [28]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (rho^2 - 1)/(2*n - 2),
    (rho^2 +rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    rho^2,
    (n-2)/(2*n-2)*rho^2+1/2*rho+1/(2*n-2),
    (rho^2+rho)/2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis(r):
    x = r^2
    y = (n-2)/(2*n-2)*r^2 +r/2+1/(2*n-2)
    z = (r^2+r)/2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=washington_basis, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=washington_permutation).lift()).factor()) 


transition matrix = 
[ 0 -1  0]
[ 0  1  1]
[ 1  0  0]
determinant =  -1

s1 = (3/8) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^6 - 14/3*n^5 + 21/2*n^4 - 39/2*n^3 + 74/3*n^2 - 95/6*n + 19/6)
s2 = (9/256) * (n^2 - 3*n + 3) * (n^2 + 3)^2 * (n^14 - 37/3*n^13 + 221/3*n^12 - 872/3*n^11 + 7711/9*n^10 - 17963/9*n^9 + 33878/9*n^8 - 5807*n^7 + 65986/9*n^6 - 67409/9*n^5 + 54266/9*n^4 - 32923/9*n^3 + 13826/9*n^2 - 3307/9*n + 79/3)
s3 = (9/4096) * (n^2 - 3*n + 3) * (n^2 + 3)^3 * (n^8 - 37/3*n^7 + 188/3*n^6 - 171*n^5 + 2410/9*n^4 - 2045/9*n^3 + 76*n^2 + 37/3*n - 19/9)

a = (1/16) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 5*n^3 + 10*n^2 - 11*n + 1)
b = (1/32) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 - 2*n - 1) * (n^2 + 3)
c = (1/32) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 8*n^3 + 16*n^2 - 16*n - 1)
d = (1/64) * (n - 1) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^3 - 11*n^2 + 19*n - 1)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-1/8) * (n - 1)^-1 * (n^2 + 3) * (n^6 - 25/2*n^5 + 83/2*n^4 - 127/2*n^3 + 83/2*n^2 - 5*n - 5)

Washington’s cyclic cubic fields, $n$ odd, good basis $\left\{\dfrac{n-2}{2n-2}\rho^2+\dfrac{n-1}{2n-2}\rho+\dfrac{1}{2n-2},-\dfrac{\rho^2-1}{2n-2}+\rho,\dfrac{\rho^2+\rho}{2}\right\}$

In [29]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (rho^2 - 1)/(2*n - 2),
    (rho^2 +rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (n-2)/(2*n-2)*rho^2+1/2*rho+1/(2*n-2),
    -(rho^2-1)/(2*n-2)+rho,
    (rho^2+rho)/2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis(r):
    x = (n-2)/(2*n-2)*r^2 +(n-1)/(2*n-2)*r+1/(2*n-2)
    y = -(r^2-1)/(2*n-2)+r
    z = (r^2+r)/2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=washington_basis, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=washington_permutation).lift()).factor()) 


transition matrix = 
[-1 -1  0]
[ 1  2  1]
[ 0 -1  0]
determinant =  -1

s1 = (1/8) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 2*n^3 + 9/2*n^2 - 13/2*n + 9/2)
s2 = (1/256) * (n^2 - 4*n + 7)^2 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^2 * (n^8 - 4*n^7 + 12*n^6 - 26*n^5 + 42*n^4 - 52*n^3 + 49*n^2 - 30*n + 11)
s3 = (1/4096) * (n^2 - 3*n + 3)^2 * (n^2 - 4*n + 7)^3 * (n^2 + 3)^3

a = (1/32) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 - 2*n + 3) * (n^2 + 3)
b = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
c = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
d = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-1/8) * (n^2 - 4*n + 7) * (n^2 + 3) * (n^3 - 7/2*n^2 + 5*n - 2)
n =  (1/8) * (n - 1) * (n^2 - 4*n + 7) * (n^2 + 3) * (n^5 - 9/2*n^4 + 21/2*n^3 - 31/2*n^2 + 16*n - 8)
p =  (1/16) * (n^2 - 4*n + 7) * (n^2 + 3) * (n^6 - 4*n^5 + 10*n^4 - 21*n^3 + 35*n^2 - 32*n + 13

Kishi's cyclic cubic fields, $n \equiv 0,2 \pmod{6} \text{ or } n \equiv 4,10 \pmod{18}$, good basis $\left\{\dfrac{n\rho^2+(n^2+n+2)\rho+1}{n^2+1},\rho,\rho^2\right\}$

In [30]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(n^2+1)*rho^2 + (n^2+n+2)/(n^2+1)*rho + 1/(n^2+1),
    rho,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (n*rho^2+(n^2+n+2)*rho+1)/(n^2+1),
    rho,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = (n*r^2+(n^2+n+2)*r+1)/(n^2+1)
    y = r
    z = r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[ 0  1  0]
[-3  0  1]
determinant =  1

s1 = (2) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 2*n^9 + 9*n^8 + 9*n^7 + 47/2*n^6 + 15/2*n^5 + 47/2*n^4 - 17/2*n^3 + 21/2*n^2 - 10*n + 5/2)
s2 = (n^2 + 1) * (n^2 + 3) * (n^3 + n^2 + 3*n - 1) * (n^4 + n^3 + 4*n^2 + 3) * (n^21 + 4*n^20 + 24*n^19 + 64*n^18 + 212*n^17 + 411*n^16 + 950*n^15 + 1379*n^14 + 2417*n^13 + 2616*n^12 + 3622*n^11 + 2807*n^10 + 3148*n^9 + 1560*n^8 + 1429*n^7 + 291*n^6 + 176*n^5 - 89*n^4 - 105*n^3 - 51*n^2 - 34*n - 8)
s3 = (n^2 + 3) * (n^2 + 1)^2 * (n^3 + n^2 + 3*n - 1)^2 * (n^4 + n^3 + 4*n^2 + 3) * (n^8 + n^7 + 6*n^6 + 2*n^5 + 8*n^4 - 3*n^3 + 2*n^2 - 1)

a = (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 5*n^4 + n^3 + 5*n^2 - 2*n + 1)
b = n * (n^2 + 3)^2 * (n^4 + n^3 + 4*n^2 + 3)
c = (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^5 + n^4 + 3*n^3 - 1)
d = (-1) * (n^2 + 3) * (n^4 + n^3 + 2*n^2 - n + 1) * (n^4 + n^3 + 4*n^2 + 3)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-2) * (n^12 + 2*n^11 +

Kishi's cyclic cubic fields, $n \equiv 34,52 \pmod{54}$, good basis $\left\{\dfrac{n\rho^2+(n^2+n+2)\rho+1}{3(n^2+1)}, \rho,\rho^2\right\}$

In [46]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(3*(n^2+1))*rho^2 + (n^2+n+2)/(3*(n^2+1))*rho + 1/(3*(n^2+1)),
    rho,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    n/(3*(n^2+1))*rho^2 + (n^2+n+2)/(3*(n^2+1))*rho + 1/(3*(n^2+1)),
    rho,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = n/(3*(n^2+1))*r^2 + (n^2+n+2)/(3*(n^2+1))*r + 1/(3*(n^2+1))
    y = r
    z = r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[ 0  1  0]
[-1  0  1]
determinant =  1

s1 = (10/9) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 14/5*n^9 + 53/5*n^8 + 93/5*n^7 + 351/10*n^6 + 423/10*n^5 + 463/10*n^4 + 391/10*n^3 + 221/10*n^2 + 62/5*n + 13/10)
s2 = (1/9) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 5*n^25 + 232/9*n^24 + 656/9*n^23 + 509/3*n^22 + 1762/9*n^21 - 937/3*n^20 - 21787/9*n^19 - 27752/3*n^18 - 207926/9*n^17 - 473455/9*n^16 - 839632/9*n^15 - 1429933/9*n^14 - 1960073/9*n^13 - 2633033/9*n^12 - 2841769/9*n^11 - 1018988/3*n^10 - 2577113/9*n^9 - 2211563/9*n^8 - 156796*n^7 - 944612/9*n^6 - 426257/9*n^5 - 210050/9*n^4 - 57856/9*n^3 - 1947*n^2 - 706/3*n - 8)
s3 = (1/81) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 3*n^17 + 97/9*n^16 + 37/3*n^15 - 16*n^14 - 1103/9*n^13 - 478*n^12 - 2647/3*n^11 - 1898*n^10 - 2165*n^9 - 3576*n^8 - 26275/9*n^7 - 12832/3*n^6 - 3069*n^5 - 32074/9*n^4 - 2383*n^3 - 4469/3*n^2 - 762*n - 73)

a = (1/9) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{-\dfrac{ n\rho^2+(2n^2+n+3)\rho+1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},\rho\right\}$

In [31]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (-n)/(2*n^2+2)*rho^2 - (2*n^2+n+3)/(2*n^2+2)*rho - 1/(2*n^2+2),
    (rho^2 + rho)/2,
    rho
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = -(n*r^2+(2*n^2+n+3)*r+1)/(2*n^2+2)
    y = (r^2+r)/2
    z = r
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[-1  0  0]
[-1  1  2]
[ 2  0 -1]
determinant =  1

s1 = (-1/4) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 3/2*n^9 + 15/2*n^8 + 4*n^7 + 59/4*n^6 - 23/4*n^5 + 19/4*n^4 - 101/4*n^3 - 53/4*n^2 - 43/2*n - 35/4)
s2 = (3/256) * n * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^25 + 7/3*n^24 + 56/3*n^23 + 18*n^22 + 287/3*n^21 - 358/3*n^20 - 111*n^19 - 5825/3*n^18 - 8416/3*n^17 - 29198/3*n^16 - 33635/3*n^15 - 74456/3*n^14 - 19907*n^13 - 97199/3*n^12 - 30989/3*n^11 - 32849/3*n^10 + 21510*n^9 + 27807*n^8 + 43201*n^7 + 42060*n^6 + 93494/3*n^5 + 70079/3*n^4 + 8814*n^3 + 12464/3*n^2 + 1175/3*n - 320)
s3 = (-9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 - 5/3*n^17 + 9*n^16 - 307/9*n^15 + 392/9*n^14 - 1751/9*n^13 + 2408/9*n^12 - 3931/9*n^11 + 9794/9*n^10 - 4205/9*n^9 + 17954/9*n^8 - 1339/3*n^7 + 9664/9*n^6 - 6181/9*n^5 - 11584/9*n^4 - 4417/9*n^3 - 4145/3*n^2 - 200*n + 125)

a = (-1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 - n^5 + 3*n^4 - 9*n^3 + n^2 - 10*n - 5)
b =

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{\dfrac{(n^2 - n + 1)\rho^2-(n^2+n+2)\rho-1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},-\rho^2\right\}$

In [32]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    ((n^2-n+1)*rho^2-(n^2+n+2)*rho-1)/(2*n^2+2),
    (rho^2 + rho)/2,
    -rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = ((n^2-n+1)*r^2-(n^2+n+2)*r-1)/(2*n^2+2)
    y = (r^2+r)/2
    z = -r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[-1  0  0]
[ 0  1  0]
[ 2  0 -1]
determinant =  1

s1 = (3/8) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 7/3*n^9 + 28/3*n^8 + 26/3*n^7 + 101/6*n^6 - 55/6*n^5 - 27/2*n^4 - 301/6*n^3 - 245/6*n^2 - 35*n - 109/6)
s2 = (9/256) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 17/3*n^25 + 34*n^24 + 346/3*n^23 + 3443/9*n^22 + 7970/9*n^21 + 17731/9*n^20 + 27943/9*n^19 + 41080/9*n^18 + 3522*n^17 + 6523/9*n^16 - 82732/9*n^15 - 183119/9*n^14 - 339619/9*n^13 - 46905*n^12 - 164279/3*n^11 - 419608/9*n^10 - 337279/9*n^9 - 180245/9*n^8 - 85396/9*n^7 - 932*n^6 + 12919/9*n^5 + 12146/9*n^4 + 8624/9*n^3 + 815/9*n^2 + 268/3*n - 4)
s3 = (9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 11/3*n^17 + 47/3*n^16 + 83/3*n^15 + 416/9*n^14 - 47/9*n^13 - 944/9*n^12 - 2855/9*n^11 - 3398/9*n^10 - 1981/9*n^9 + 306*n^8 + 6691/9*n^7 + 2392/3*n^6 + 3331/9*n^5 - 680/9*n^4 - 1397/9*n^3 - 121/3*n^2 + 8*n + 1)

a = (1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 3*n^4 - n^3 - 3*n^2 -

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{\dfrac{(n^2 + n + 1)\rho^2+(n^2 + n + 2)\rho+1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},\rho^2\right\}$

In [33]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    ((n^2+n+1)*rho^2+(n^2+n+2)*rho+1)/(2*n^2+2),
    (rho^2 + rho)/2,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = ((n^2+n+1)*r^2+(n^2+n+2)*r+1)/(2*n^2+2)
    y = (r^2+r)/2
    z = r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[ 0  1  0]
[-1  0  1]
determinant =  1

s1 = (3/8) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 5/3*n^9 + 22/3*n^8 + 16/3*n^7 + 101/6*n^6 + 7/2*n^5 + 143/6*n^4 + 23/6*n^3 + 175/6*n^2 + 13/3*n + 95/6)
s2 = (9/256) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 13/3*n^25 + 82/3*n^24 + 238/3*n^23 + 2531/9*n^22 + 1826/3*n^21 + 13907/9*n^20 + 23827/9*n^19 + 47720/9*n^18 + 7582*n^17 + 12683*n^16 + 145556/9*n^15 + 209989/9*n^14 + 28469*n^13 + 35263*n^12 + 381427/9*n^11 + 129640/3*n^10 + 445981/9*n^9 + 39055*n^8 + 40468*n^7 + 203392/9*n^6 + 60949/3*n^5 + 20630/3*n^4 + 15736/3*n^3 + 6407/9*n^2 + 1348/3*n - 4)
s3 = (9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 7/3*n^17 + 35/3*n^16 + 43/3*n^15 + 368/9*n^14 + 209/9*n^13 + 188/3*n^12 + 193/9*n^11 + 458/9*n^10 + 491/9*n^9 - 290/9*n^8 + 329/3*n^7 - 1352/9*n^6 - 509/9*n^5 - 2684/9*n^4 - 2845/9*n^3 - 553/3*n^2 - 308*n - 71)

a = (1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 3*n^4 - n^3 + n^2 

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{\dfrac{n\rho^2+(n+1)\rho+1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},-\rho\right\}$

In [34]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (n*rho^2+(n+1)*rho+1)/(2*n^2+2),
    (rho^2 + rho)/2,
    -rho
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = (n*r^2+(n+1)*r+1)/(2*n^2+2)
    y = (r^2+r)/2
    z = -r
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[-1  1 -2]
[-1  0  1]
determinant =  1

s1 = (-1/4) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 7/2*n^9 + 23/2*n^8 + 23*n^7 + 147/4*n^6 + 201/4*n^5 + 191/4*n^4 + 195/4*n^3 + 115/4*n^2 + 37/2*n + 21/4)
s2 = (3/256) * n * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^25 + 23/3*n^24 + 136/3*n^23 + 190*n^22 + 661*n^21 + 5806/3*n^20 + 4881*n^19 + 32875/3*n^18 + 21632*n^17 + 117062/3*n^16 + 62623*n^15 + 279124/3*n^14 + 123757*n^13 + 458897/3*n^12 + 169641*n^11 + 521671/3*n^10 + 479266/3*n^9 + 400541/3*n^8 + 99281*n^7 + 195296/3*n^6 + 110446/3*n^5 + 51943/3*n^4 + 18274/3*n^3 + 1536*n^2 - 97/3*n - 96)
s3 = (-9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 23/3*n^17 + 37*n^16 + 1189/9*n^15 + 3272/9*n^14 + 7529/9*n^13 + 4820/3*n^12 + 8015/3*n^11 + 3906*n^10 + 14921/3*n^9 + 51406/9*n^8 + 50423/9*n^7 + 43904/9*n^6 + 32443/9*n^5 + 6644/3*n^4 + 9847/9*n^3 + 309*n^2 - 36*n - 27)

a = (-1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + 3*n^5 + 7*n^4 + 11*n^3 +

Kishi's cyclic cubic fields, $n \equiv 7,25 \pmod{54}$, good basis $\left\{\dfrac{A}{6},\dfrac{\rho^2+\rho}{2},\rho^2\right\}$

In [36]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(6*(n^2+1))*rho^2 + (n^2+n+2)/(6*(n^2+1))*rho + 1/(6*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (3*n^2+n+3)/(6*(n^2+1))*rho^2 + (n^2+n+2)/(6*(n^2+1))*rho + 1/(6*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = (3*n^2+n+3)/(6*(n^2+1))*r^2 + (n^2+n+2)/(6*(n^2+1))*r + 1/(6*(n^2+1))
    y = (r^2+r)/2
    z = r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[1 0 0]
[0 1 0]
[0 0 1]
determinant =  1

s1 = (5/24) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 13/5*n^9 + 124/15*n^8 + 62/5*n^7 + 533/30*n^6 + 553/30*n^5 + 271/30*n^4 + 371/30*n^3 - 149/30*n^2 + 59/15*n - 109/30)
s2 = (1/256) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 43/9*n^25 + 614/27*n^24 + 1642/27*n^23 + 3473/27*n^22 + 4262/27*n^21 - 13117/81*n^20 - 29395/27*n^19 - 344248/81*n^18 - 684926/81*n^17 - 1477837/81*n^16 - 1982540/81*n^15 - 3162047/81*n^14 - 2988995/81*n^13 - 1280059/27*n^12 - 2396645/81*n^11 - 2681456/81*n^10 - 853247/81*n^9 - 993277/81*n^8 + 32956/81*n^7 - 40828/27*n^6 + 105287/81*n^5 + 26962/81*n^4 + 22256/81*n^3 + 1861/27*n^2 + 332/27*n - 4/9)
s3 = (1/36864) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 25/9*n^17 + 167/27*n^16 + 17/9*n^15 - 800/27*n^14 - 1709/27*n^13 - 13336/81*n^12 - 4375/81*n^11 - 2998/9*n^10 - 18341/81*n^9 - 87014/81*n^8 - 43549/81*n^7 - 13576/27*n^6 - 4727/27*n^5 - 3584/81*n^4 + 257/3*n^3 + 815/27*n^2 + 16/3*n + 

Kishi's cyclic cubic fields, $n \equiv 7,25 \pmod{54}$, good basis $\left\{\dfrac{A}{6},\dfrac{-n\rho^2+(2n^2-n+1)\rho-1}{6(n^2+1)},\dfrac{-n\rho^2-(n^2+n+2)\rho-1}{3(n^2+1)}\right\}$

In [40]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    ((3*n^2+n+3)*rho^2+(n^2+n+2)*rho+1)/(6*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    ((3*n^2+n+3)*rho^2+(n^2+n+2)*rho+1)/(6*(n^2+1)),
    (-n*rho^2+(2*n^2-n+1)*rho-1)/(6*(n^2+1)),
    (-n)/(3*(n^2+1))*rho^2 - (n^2+n+2)/(3*(n^2+1))*rho - 1/(3*(n^2+1))
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = (3*n^2+n+3)/(6*(n^2+1))*r^2 + (n^2+n+2)/(6*(n^2+1))*r + 1/(6*(n^2+1))
    y = (-n)/(6*(n^2+1))*r^2 + (2*n^2-n+1)/(6*(n^2+1))*r - 1/(6*(n^2+1))
    z = (-n)/(3*(n^2+1))*r^2 - (n^2+n+2)/(3*(n^2+1))*r - 1/(3*(n^2+1))
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1 -1 -2]
[ 0  1  0]
[ 0  0  1]
determinant =  1

s1 = (-1/12) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 13/6*n^9 + 9*n^8 + 32/3*n^7 + 283/12*n^6 + 169/12*n^5 + 91/4*n^4 + 9/4*n^3 + 27/4*n^2 - 25/6*n - 13/12)
s2 = (1/768) * (n^2 + 3) * (n^3 + n^2 + 3*n - 1) * (n^4 + n^3 + 4*n^2 + 3) * (n^23 + 8/3*n^22 + 155/9*n^21 + 604/27*n^20 + 1958/27*n^19 - 685/9*n^18 - 7006/27*n^17 - 43964/27*n^16 - 91907/27*n^15 - 76495/9*n^14 - 365131/27*n^13 - 624113/27*n^12 - 779528/27*n^11 - 1001239/27*n^10 - 984061/27*n^9 - 107603/3*n^8 - 735835/27*n^7 - 181516/9*n^6 - 305845/27*n^5 - 52982/9*n^4 - 59563/27*n^3 - 5915/9*n^2 - 1090/9*n - 26/3)
s3 = (-1/36864) * (n^2 + 3) * (n^3 + n^2 + 3*n - 1)^2 * (n^4 + n^3 + 4*n^2 + 3) * (n^12 - n^11 + 40/9*n^10 - 124/9*n^9 - 311/81*n^8 - 3916/81*n^7 - 62/3*n^6 - 3730/81*n^5 - 47/81*n^4 + 383/27*n^3 + 590/27*n^2 + 74/3*n + 37/9)

a = (-1/144) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 41/9*n^4 - 1/9*n^3 + 23/9*n^2 - 4*n - 1)
b = (1/576

Kishi's cyclic cubic fields, $n \equiv 16 \pmod{54}$, good basis $\left\{\dfrac{\rho^2-\rho}{3},\rho^2,-\dfrac{A}{9}\right\}$

In [49]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(9*n^2+9)*rho^2 +(n^2+n+2)/(9*n^2+9)* rho  +1/(9*n^2+9),
    (2*rho^2+rho)/3 ,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (rho^2-rho)/3,
    rho^2,
    -(3*n^2+n+3)/(9*n^2+9)*rho^2 -(n^2+n+2)/(9*n^2+9)* rho  -1/(9*n^2+9)
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x =  (r^2-r)/3
    y =  r^2
    z = -(3*n^2+n+3)/(9*n^2+9)*r^2 -(n^2+n+2)/(9*n^2+9)* r  -1/(9*n^2+9)
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 0  0 -1]
[-1  0  0]
[ 1  1  0]
determinant =  1

s1 = (80/729) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 109/40*n^9 + 981/80*n^8 + 1917/80*n^7 + 4357/80*n^6 + 5987/80*n^5 + 8551/80*n^4 + 951/10*n^3 + 7227/80*n^2 + 1661/40*n + 537/20)
s2 = (64/59049) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 59/12*n^25 + 659/24*n^24 + 1981/24*n^23 + 32477/144*n^22 + 25775/72*n^21 + 27737/192*n^20 - 359449/192*n^19 - 5144783/576*n^18 - 5272549/192*n^17 - 3195091/48*n^16 - 26675711/192*n^15 - 987627/4*n^14 - 231448417/576*n^13 - 80129731/144*n^12 - 422215133/576*n^11 - 76696781/96*n^10 - 165480799/192*n^9 - 104751971/144*n^8 - 370019797/576*n^7 - 231398095/576*n^6 - 2583077/9*n^5 - 1938993/16*n^4 - 12954955/192*n^3 - 247687/16*n^2 - 93119/16*n - 20751/64)
s3 = (64/43046721) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 35/12*n^17 + 117/8*n^16 + 199/8*n^15 + 4475/144*n^14 - 1649/24*n^13 - 223505/576*n^12 - 77377/64*n^11 - 843817/288*n^10 - 859339/144*n^9 - 6735383/576*n

Kishi's cyclic cubic fields, $n \equiv 16 \pmod{54}$, good basis $\left\{\dfrac{A}{9},\dfrac{n\rho^2+(4n^2+n+5)\rho+1}{9(n^2+1)},\dfrac{-n\rho^2+(n^2+n+2)\rho+1}{3(n^2+1)}\right\}$

In [52]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    ((3*n^2+n+3)*rho^2+(n^2+n+2)*rho+1)/(9*(n^2+1)),
    (2*rho^2+rho)/3,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    ((3*n^2+n+3)*rho^2+(n^2+n+2)*rho+1)/(9*(n^2+1)),
    ((n)*rho^2+(4*n^2+n+5)*rho+1)/(9*(n^2+1)),
    ((-n)*rho^2-(n^2+n+2)*rho-1)/(3*(n^2+1))
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x =  ((3*n^2+n+3)*r^2+(n^2+n+2)*r+1)/(9*(n^2+1))
    y =  ((n)*r^2+(4*n^2+n+5)*r+1)/(9*(n^2+1))
    z =  ((-n)*r^2-(n^2+n+2)*r-1)/(3*(n^2+1))
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  1 -3]
[ 0  1  0]
[ 0 -1  1]
determinant =  1

s1 = (-17/729) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 175/51*n^9 + 631/51*n^8 + 1145/51*n^7 + 2086/51*n^6 + 2195/51*n^5 + 2464/51*n^4 + 429/17*n^3 + 941/51*n^2 + 44/51*n - 13/17)
s2 = (8/59049) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 31/4*n^25 + 3505/72*n^24 + 44675/216*n^23 + 488983/648*n^22 + 1443425/648*n^21 + 470893/81*n^20 + 8425397/648*n^19 + 5638345/216*n^18 + 7457569/162*n^17 + 47566765/648*n^16 + 8420294/81*n^15 + 86398459/648*n^14 + 32907647/216*n^13 + 33921205/216*n^12 + 10352579/72*n^11 + 8449051/72*n^10 + 27291661/324*n^9 + 5663725/108*n^8 + 18207575/648*n^7 + 7936513/648*n^6 + 2801255/648*n^5 + 36637/36*n^4 + 2953/24*n^3 - 299/24*n^2 - 161/18*n - 9/8)
s3 = (-64/43046721) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 77/12*n^17 + 2305/72*n^16 + 22933/216*n^15 + 23647/81*n^14 + 2446555/3888*n^13 + 2977769/2592*n^12 + 4452331/2592*n^11 + 33900115/15552*n^10 + 11879333/5184*n^9 + 104647

Kishi's cyclic cubic fields, $n \equiv 43 \pmod{54}$, good basis $\left\{\rho^2,\dfrac{-\rho^2+\rho}{6},-\dfrac{A}{6}\right\}$

In [43]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(18*n^2+18)*rho^2 +(n^2+n+2)/(18*n^2+18)* rho  +1/(18*n^2+18),
    (5*rho^2+rho)/6 ,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    rho^2,
    (-rho^2+rho)/6,
    -(3*n^2+n+3)/(18*n^2+18)*rho^2 -(n^2+n+2)/(18*n^2+18)* rho  -1/(18*n^2+18)
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x =  r^2
    y =  (-r^2+r)/6
    z = -(3*n^2+n+3)/(18*n^2+18)*r^2 -(n^2+n+2)/(18*n^2+18)* r  -1/(18*n^2+18)
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 0  0 -1]
[ 0  1  0]
[ 1 -1  0]
determinant =  1

s1 = (-175/5832) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 487/175*n^9 + 2178/175*n^8 + 864/35*n^7 + 3917/70*n^6 + 3887/50*n^5 + 38791/350*n^4 + 34851/350*n^3 + 32931/350*n^2 + 1531/35*n + 1947/70)
s2 = (1225/15116544) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 523/105*n^25 + 2918/105*n^24 + 8794/105*n^23 + 2513279/11025*n^22 + 3878578/11025*n^21 + 342953/3675*n^20 - 7893979/3675*n^19 - 108233348/11025*n^18 - 110613382/3675*n^17 - 267030247/3675*n^16 - 559136672/3675*n^15 - 331757799/1225*n^14 - 4874523559/11025*n^13 - 1356514529/2205*n^12 - 8949390341/11025*n^11 - 93647096/105*n^10 - 3531292573/3675*n^9 - 1811538397/2205*n^8 - 7944908056/11025*n^7 - 5075444956/11025*n^6 - 3565651541/11025*n^5 - 173636478/1225*n^4 - 39972508/525*n^3 - 23007601/1225*n^2 - 228832/35*n - 77928/175)
s3 = (-1225/176319369216) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 313/105*n^17 + 519/35*n^16 + 883/35*n^15 + 42524/1102